In [1]:
import sys
import os
from SciExpeM_API.SciExpeM import SciExpeM
from SciExpeM_API.Models import *

sys.path.append(r"C:\Users\lpratalimaffei\Desktop\OneDrive - Politecnico di Milano\Luna\Universita\PhD\SCIEXPEM") # path to extract_data.py
from extract_data import readdata
from extract_data import readprofile

my_sciexpem = SciExpeM(username='yourid', password='yourpw')

LABEL_DCT = {'temperature' : 'T', 'time' : 't', 'length' : 'l', 'pressure' : 'p', 'volume' : 'V'}


### FilePaper
 - description*
 - reference_doi*
 - author*
 - title*
 - year*
 - volume
 - page
 - journal

In [2]:
#path where you find os input file and data
datapath = r'D:\POLI\OPENSMOKE\CPD\PYROLYSIS\PFR_Djokic2014'

In [3]:
file_paper = FilePaper(reference_doi="10.1016/J.COMBUSTFLAME.2014.04.013", 
                       author="Van Geem, Kevin Marcel, 0000-0003-4191-4960; Ranzi, Eliseo, 0000-0002-1395-6074; Frassoldati, Alessio, 0000-0003-0264-438; Djokic, Marko, 0000-0002-9150-1964; Cavallotti, Carlo; Marin, Guy B.",                       
                       title="An Experimental And Kinetic Modeling Study Of Cyclopentadiene Pyrolysis: First Growth Of Polycyclic Aromatic Hydrocarbons",
                       year=2014,
                       description="Van Geem, Kevin Marcel, 0000-0003-4191-4960; Ranzi, Eliseo, 0000-0002-1395-6074; Frassoldati, Alessio, 0000-0003-0264-438; Djokic, Marko, 0000-0002-9150-1964; Cavallotti, Carlo; Marin, Guy B. - Combustion And Flame, 2014, (161), 2739-2751"
                       )

### Common properties

- name
- units
- value
- source_type

In [4]:
c1 = CommonProperty(name='pressure', units='atm', value='1.7', source_type='reported')
c2 = CommonProperty(name='length', units='mm', value='1475', source_type='reported')
c3 = CommonProperty(name='volume', units='cm3', value='10252.39128', source_type='calculated')
# if parametric analysis, report only the common feature, see other examples
commonprop = [c1, c2, c3]

### OpenSMOKE input file if available

In [33]:
# WARNING LIST OF SPECIES MUST HAVE THE NAMES OF THE MECH YOU ARE GOING TO SIMULATE - OTHERWISE, NEED REPLACEMENT
with open(os.path.join(datapath, 'input.dic')) as f:
    inputstr = f.read()  

In [ ]:
c1 = CommonProperty(name='pressure', units='atm', value='1.7', source_type='reported')
c2 = CommonProperty(name='length', units='mm', value='1475', source_type='reported')
c3 = CommonProperty(name='volume', units='cm3', value='10252.39128', source_type='calculated')
# if parametric analysis, report only the common feature, see other examples
commonprop = [c1, c2, c3]

In [34]:
with open(os.path.join(datapath, 'input.dic')) as f:
    # now search for profiles
    for line in f:
        if 'ListOfProfiles' in line and '//' not in line.split('ListOfProfiles')[0]:
            print('reminder if sth looks weird - ListOfProfiles must be all in the same row')
            csv_profiles = line.split('ListOfProfiles')[1].strip().split(';')[0].split()
            print(csv_profiles)
            csv_paths = [os.path.join(datapath, file) for file in csv_profiles]

reminder if sth looks weird - ListOfProfiles must be all in the same row
['1.csv', '2.csv', '3.csv', '4.csv', '5.csv']


### Initial Species

- name
- units
- value
- source_type
- configuration

In [35]:
# THIS REFERS TO THE SIMULATION
# PREMIXED IS DEFAULT AND MUST BE INDICATED UNLESS IT'S A CF FLAME
species = ['C5H6', 'N2']
composition = ['0.089795918', '0.910204082']
comp_unit = 'mole fraction'
srctype = 'calculated'
config = 'premixed'
##############################
################# do not edit
inspecies = []
for i, s in enumerate(species):
    si = my_sciexpem.filterDatabase(model_name='Specie', preferredKey=s)[0]
    ii = InitialSpecie(name=species[i], 
                    specie=si, 
                    units=comp_unit, 
                    value=composition[i], 
                    source_type=srctype, 
                    configuration=config)

    inspecies.append(ii)

### Data columns

- name
- label (not comuplsory)
- species_object
- units
- data
- dg_id 
- dg_label
- source_type


In [36]:
# read species profiles
datafile =  open(os.path.join(datapath, 'data_HD.txt'))
df_data = readdata(datafile, delzero = False) # DO NOT DELETE ZERO VALUES - REACTORS WITH PROFILES MUST ALL HAVE THE SAME X AXIS!
# process data

srctype = 'calculated'
label = 'experimental_data'
x = ['temperature', 'K']
#y = ['concentration', 'mol/cm3']
#x = ['distance', 'cm']
y = ['composition', 'mole fraction']
TINF = 3500
TSUP = 0
datacols = []
dgs = []
#################### DO NOT EDIT################
for dg, df in df_data.items():
    # x axis
    x_data = list(df.index)
    x_datacol = DataColumn(name=x[0], units=x[1], dg_id=dg, dg_label=label, data=x_data,
                           source_type=srctype)
    datacols.append(x_datacol)
    len_dg = len(x_data)
    if x[0] == 'temperature' and min(x_data) < TINF:
        TINF = min(x_data)
    if x[0] == 'temperature' and max(x_data) > TSUP:
        TSUP = max(x_data)
    # append dg to dgs to count
    dgs.append(dg)
    # y
    for y_name in df.columns:
        # species
        # print(y_name)
        spobj = []
        for sp in y_name.split('+'):
            spobj_temp = my_sciexpem.filterDatabase(
                model_name='Specie', preferredKey=sp)
            if len(spobj_temp) == 0:
                print(
                    'species {} not as preferred name: alternative names searched'.format(sp))
                spobj_temp = my_sciexpem.filterDatabase(
                    model_name='Specie', names__contains=[sp])
                
            if len(spobj_temp) == 0:
                print('warning: species {} not found and not inserted in the columns - please check it\'s what you want'.format(y_name))
            else:
                spobj += spobj_temp
        # print(spobj)
        # final data
        if len(spobj) > 0:
            y_datacol = DataColumn(name=y[0], units=y[1], dg_id=dg, dg_label=label,
                                data=list(df.loc[x_data][y_name]), source_type=srctype,
                                species_object=spobj)

            datacols.append(y_datacol)


species CH3C6H4C2H5 not as preferred name: alternative names searched
species C9H8 not as preferred name: alternative names searched
species C9H7CH3 not as preferred name: alternative names searched
species C10H7CH3-1 not as preferred name: alternative names searched
species C10H7CH3-2 not as preferred name: alternative names searched
species C12H10 not as preferred name: alternative names searched
species MECHRYS not as preferred name: alternative names searched
species C21H14 not as preferred name: alternative names searched
species C18H12 not as preferred name: alternative names searched


In [37]:
#PROFILES
srctype = 'reported'

########## DO NOT EDIT ###########################
init_cond1 = []
init_cond2 = []
profiles = []
# first datagroup number
dgs.sort()
dg0 = int(dgs[-1].split('dg')[1])
if len(dgs) > 1:
    print('warning - more than 1 datagroup found - there should be just one, where each x corresponds to a different profile output')
# loop over csv profiles
for i, csv_file in enumerate(csv_paths):
    profile_info = readprofile(csv_file)
    # append initial conditions
    init_cond1.append(profile_info['init-cond1']['value'])
    init_cond2.append(profile_info['init-cond2']['value'])
    # find dg label
    # tipologie di profili: V-t, p-t, T-t, T-length
    dg_label = LABEL_DCT[profile_info['y']['name']] + '-' + LABEL_DCT[profile_info['x']['name']] + ' history'
    # append profile info for x and y
    for ii in ['y', 'x']:
        profile_datacol = DataColumn(name=profile_info[ii]['name'], 
                    units = profile_info[ii]['units'], 
                    dg_id ='dg' + str(dg0 + i + 2),
                    dg_label = dg_label,
                    data = list(profile_info[ii]['value']),
                    source_type = srctype,
                    data_group_profile=[str(i + 1)])
        
        profiles.append(profile_datacol)

# INITIAL CONDITIONS: do at the end, but datagroup number will be dg0 + 1
init_cond = {'init-cond1': init_cond1, 'init-cond2': init_cond2}
for ii in ['init-cond1', 'init-cond2']:
    profile_initcond = DataColumn(name=profile_info[ii]['name'], 
                units = profile_info[ii]['units'], 
                dg_id ='dg' + str(dg0 + 1),
                dg_label = 'initial_condition',
                data = init_cond[ii],
                source_type = srctype)    
    datacols.append(profile_initcond)

# finally append the profiles
datacols += profiles

### Assembly the experiment

In [38]:
print(TINF,TSUP)

873.0 1120.0


In [39]:
e = Experiment(reactor='flow reactor', 
               experiment_type='outlet concentration measurement', #NO CAPITAL LETTERS HERE
               file_paper=file_paper, 
               data_columns=datacols, 
               initial_species=inspecies, 
               common_properties=commonprop,
               os_input_file=inputstr,
               t_inf = TINF, t_sup = TSUP,
               p_inf = 1.7, p_sup = 1.7,
               phi_inf = 100., phi_sup = 100.,
               fuels = ['C5H6'],
               comment = 'original data reported in yield (weight%) and converted to mole fractions'
               #comment = 'C4H2 plot unclear, order might be 1e6 or 1e8 (person who digitized assumed 1e8); bath gas varies with experimental run (Ne, He), but should not affect the results'
               )

In [31]:
e.serialize()

{'data_columns': [{'name': 'temperature',
   'units': 'K',
   'data': [873.0, 973.0, 1020.0, 1070.0, 1120.0],
   'dg_id': 'dg1',
   'dg_label': 'experimental_data',
   'label': None,
   'source_type': 'calculated',
   'data_group_profile': None,
   'uncertainty_kind': None,
   'uncertainty_bound': None,
   'species_object': None,
   'uncertainty_reference': None},
  {'name': 'composition',
   'units': 'mole fraction',
   'data': [0.00114, 0.00344, 0.0086, 0.0462, 0.0692],
   'dg_id': 'dg1',
   'dg_label': 'experimental_data',
   'label': None,
   'source_type': 'calculated',
   'data_group_profile': None,
   'uncertainty_kind': None,
   'uncertainty_bound': None,
   'species_object': [1],
   'uncertainty_reference': None},
  {'name': 'composition',
   'units': 'mole fraction',
   'data': [0.000503, 0.00195, 0.00598, 0.0191, 0.0243],
   'dg_id': 'dg1',
   'dg_label': 'experimental_data',
   'label': None,
   'source_type': 'calculated',
   'data_group_profile': None,
   'uncertainty_kin

### Send Experiment

In [40]:
my_sciexpem.insertElement(e, verbose=True)
# QUANDO HAI COMPLETATO IL CARICAMENTO SU SCIEXPEM TU PUOI FARE IL DOWNLOAD DI UN JSON; 

Experiment element inserted successfully.
